In [1]:
import bloqade.stim
import bloqade.tsim

import warnings

import cirq
import numpy as np
import matplotlib.pyplot as plt
import bloqade.cirq_utils as utils
from cirq.contrib.svg import SVGCircuit

from bloqade import squin, cirq_utils

warnings.filterwarnings("ignore")

@squin.kernel
def main():
    q_data = squin.qalloc(7)
    # squin.u3(theta, phi, 0, q_data[6])

    # apply sqrt_y_adj to first 5 gates
    for i in range(6):
        squin.sqrt_y_adj(q_data[i])

    # apply cz to pairs
    squin.cz(q_data[1], q_data[2])
    squin.cz(q_data[3], q_data[4])
    squin.cz(q_data[5], q_data[6])

    # apply sqrt_y gate to injection state
    squin.sqrt_y(q_data[6])

    # apply cz to more pairs
    squin.cz(q_data[0], q_data[3])
    squin.cz(q_data[2], q_data[5])
    squin.cz(q_data[4], q_data[6])

    # apply sqrt_y gates
    for i in range(2, 7):
        squin.sqrt_y(q_data[i])

    # apply MORE cz gates
    squin.cz(q_data[0], q_data[1])
    squin.cz(q_data[2], q_data[3])
    squin.cz(q_data[4], q_data[5])

    squin.sqrt_y(q_data[1])
    squin.sqrt_y(q_data[2])
    squin.sqrt_y(q_data[4])

    """
    Encode aux |+>
    """
    q_aux_1 = squin.qalloc(7)
    squin.h(q_aux_1[6])

    # apply sqrt_y_adj to first 5 gates
    for i in range(6):
        squin.sqrt_y_adj(q_aux_1[i])

    # apply cz to pairs
    squin.cz(q_aux_1[1], q_aux_1[2])
    squin.cz(q_aux_1[3], q_aux_1[4])
    squin.cz(q_aux_1[5], q_aux_1[6])

    # apply sqrt_y gate to injection state
    squin.sqrt_y(q_aux_1[6])

    # apply cz to more pairs
    squin.cz(q_aux_1[0], q_aux_1[3])
    squin.cz(q_aux_1[2], q_aux_1[5])
    squin.cz(q_aux_1[4], q_aux_1[6])

    # apply sqrt_y gates
    for i in range(2, 7):
        squin.sqrt_y(q_aux_1[i])

    # apply MORE cz gates
    squin.cz(q_aux_1[0], q_aux_1[1])
    squin.cz(q_aux_1[2], q_aux_1[3])
    squin.cz(q_aux_1[4], q_aux_1[5])

    squin.sqrt_y(q_aux_1[1])
    squin.sqrt_y(q_aux_1[2])
    squin.sqrt_y(q_aux_1[4])


    """
    Encode aux |0>
    """
    q_aux_2 = squin.qalloc(7)

    # apply sqrt_y_adj to first 5 gates
    for i in range(6):
        squin.sqrt_y_adj(q_aux_2[i])

    # apply cz to pairs
    squin.cz(q_aux_2[1], q_aux_2[2])
    squin.cz(q_aux_2[3], q_aux_2[4])
    squin.cz(q_aux_2[5], q_aux_2[6])

    # apply sqrt_y gate to injection state
    squin.sqrt_y(q_aux_2[6])

    # apply cz to more pairs
    squin.cz(q_aux_2[0], q_aux_2[3])
    squin.cz(q_aux_2[2], q_aux_2[5])
    squin.cz(q_aux_2[4], q_aux_2[6])

    # apply sqrt_y gates
    for i in range(2, 7):
        squin.sqrt_y(q_aux_2[i])

    # apply MORE cz gates
    squin.cz(q_aux_2[0], q_aux_2[1])
    squin.cz(q_aux_2[2], q_aux_2[3])
    squin.cz(q_aux_2[4], q_aux_2[5])

    squin.sqrt_y(q_aux_2[1])
    squin.sqrt_y(q_aux_2[2])
    squin.sqrt_y(q_aux_2[4])

    for i in range(7):
        squin.cx(q_data[i], q_aux_1[i])

    squin.broadcast.measure(q_aux_1, key='result')

    for i in range(7):
        squin.cx(q_aux_2[i], q_data[i])
        squin.h(q_aux_2[i])

    squin.broadcast.measure(q_aux_2, key='result')

    squin.broadcast.measure(q_data, key='result')

# plotting circuit diagram (plotting is nicer on Tsim)
tsim_circ = bloqade.tsim.Circuit(main)
tsim_circ.diagram(height=400)

# sampling using Stim (Clifford-only sampling is faster on Stim)
shots = 100
stim_circ = bloqade.stim.Circuit(main)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=shots)

print(f"Sampled {shots} shots")
print(f"Sample shape: {samples.shape}")

Sampled 100 shots
Sample shape: (100, 21)


In [2]:
tsim_circ.diagram(height=400)

In [3]:
aux_1_results = samples[:, 0:7]
aux_2_results = samples[:, 7:14]
data_results = samples[:, 14:21]

In [4]:
stabilizer1 = []
stabilizer2 = []
stabilizer3 = []

for shot in range(shots):
    sample = aux_1_results[shot]
    stab1 = sample[0] ^ sample[1] ^ sample[2] ^ sample[3]
    stabilizer1.append(stab1)
    stab2 = sample[2] ^ sample[3] ^ sample[4] ^ sample[6]
    stabilizer2.append(stab2)
    stab3 = sample[1] ^ sample[2] ^ sample[4] ^ sample[5]
    stabilizer3.append(stab3)

In [5]:
stabilizer1

[np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.

In [6]:
stabilizer2

[np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.

In [7]:
stabilizer3

[np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.False_,
 np.

In [8]:
def false_indices(stabilizer1, stabilizer2, stabilizer3):
    return [
        i for i, (a, b, c) in enumerate(zip(stabilizer1, stabilizer2, stabilizer3))
        if not a and not b and not c
    ]


In [11]:
results = false_indices(stabilizer1, stabilizer2, stabilizer3)

In [ ]:
logical_d = []

for i in results:
    sample = data_results[i]
    log = sample[0] ^ sample[1] ^ sample[5]
    logical_d.append(log)

num_zeros = logical_d.count(False)
num_ones = logical_d.count(True)

In [16]:
print(num_zeros)

NameError: name 'num_zeros' is not defined

In [14]:
stabilizer1_a = []
stabilizer2_a= []
stabilizer3_a = []

for shot in range(shots):
    sample = aux_2_results[shot]
    stab1 = sample[0] ^ sample[1] ^ sample[2] ^ sample[3]
    stabilizer1.append(stab1)
    stab2 = sample[2] ^ sample[3] ^ sample[4] ^ sample[6]
    stabilizer2.append(stab2)
    stab3 = sample[1] ^ sample[2] ^ sample[4] ^ sample[5]
    stabilizer3.append(stab3)

In [15]:
results3 = print(false_indices(stabilizer1_a, stabilizer2_a, stabilizer3_a))

[]
